<a href="https://colab.research.google.com/github/TallesMadi/DIO_machine_learning/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
!curl -L "https://github.com/datitran/raccoon_dataset/archive/refs/heads/master.zip" -o raccoon_dataset.zip
!unzip -q raccoon_dataset.zip -d raccoon_dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 46.9M    0 46.9M    0     0  7509k      0 --:--:--  0:00:06 --:--:-- 9208k


In [3]:
import os
import shutil

In [4]:
os.makedirs("datasets/raccoon/images/train", exist_ok=True)
os.makedirs("datasets/raccoon/images/val", exist_ok=True)
os.makedirs("datasets/raccoon/labels/train", exist_ok=True)
os.makedirs("datasets/raccoon/labels/val", exist_ok=True)

In [5]:
images = [f for f in os.listdir("raccoon_dataset/raccoon_dataset-master/images") if f.endswith(".jpg")]
split = int(0.8 * len(images))

for i, img in enumerate(images):
    label = img.replace(".jpg", ".xml")
    if i < split:
        shutil.copy(f"raccoon_dataset/raccoon_dataset-master/images/{img}", "datasets/raccoon/images/train/")
        shutil.copy(f"raccoon_dataset/raccoon_dataset-master/annotations/{label}", "datasets/raccoon/labels/train/")
    else:
        shutil.copy(f"raccoon_dataset/raccoon_dataset-master/images/{img}", "datasets/raccoon/images/val/")
        shutil.copy(f"raccoon_dataset/raccoon_dataset-master/annotations/{label}", "datasets/raccoon/labels/val/")

In [6]:
!pip install pascal-voc-writer lxml

import xml.etree.ElementTree as ET

In [7]:
def convert_voc_to_yolo(voc_path, yolo_path, img_size=(640, 640)):
    tree = ET.parse(voc_path)
    root = tree.getroot()

    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)

    with open(yolo_path, "w") as f:
        for obj in root.findall("object"):
            cls = obj.find("name").text
            cls_id = 0
            xmlbox = obj.find("bndbox")
            xmin = int(xmlbox.find("xmin").text)
            xmax = int(xmlbox.find("xmax").text)
            ymin = int(xmlbox.find("ymin").text)
            ymax = int(xmlbox.find("ymax").text)

            x = (xmin + xmax) / 2.0 / w
            y = (ymin + ymax) / 2.0 / h
            bw = (xmax - xmin) / w
            bh = (ymax - ymin) / h
            f.write(f"{cls_id} {x} {y} {bw} {bh}\n")

In [8]:
for subset in ["train", "val"]:
    xml_folder = f"datasets/raccoon/labels/{subset}"
    for file in os.listdir(xml_folder):
        if file.endswith(".xml"):
            voc_path = os.path.join(xml_folder, file)
            yolo_path = voc_path.replace(".xml", ".txt")
            convert_voc_to_yolo(voc_path, yolo_path)
            os.remove(voc_path)

In [9]:
with open("datasets/raccoon/data.yaml", "w") as f:
    f.write(
        "train: /content/datasets/raccoon/images/train\n"
        "val: /content/datasets/raccoon/images/val\n"
        "nc: 1\n"
        "names: ['raccoon']\n"
    )


In [ ]:
model = YOLO("yolov8n.pt")
model.train(
    data="datasets/raccoon/data.yaml",
    epochs=10,
    imgsz=640,
    batch=16
)

Ultralytics 8.3.183 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/raccoon/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True,

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

train: Scanning /content/datasets/raccoon/labels/train... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<00:00, 307.77it/s]

train: New cache created: /content/datasets/raccoon/labels/train.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/10         0G     0.9226      1.602      1.512         18        640: 100%|██████████| 10/10 [02:06<00:00, 12.66s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.32s/it]

                   all         40         43      0.937      0.697      0.808      0.452



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
metrics = model.val()
print(metrics)